# **Error estándar(SE) y Bootstrap**

##  Método Bootstrap

El **Bootstrap** es un procedimiento de remuestreo que permite aproximar la distribución de un estadístico de interés sin necesidad de supuestos fuertes sobre la población.

1. A partir de una muestra original de tamaño $n$, se generan $B$ nuevas muestras (con reemplazo).
2. En cada remuestra se calcula el estadístico de interés (ej. media, mediana, varianza).
3. La colección de estos valores se usa para aproximar la distribución del estadístico.

Formalmente, si $\hat{\theta}$ es el estimador y $\hat{\theta}^*_b$ es el estadístico en la $b$-ésima muestra bootstrap, entonces:

$$
\{ \hat{\theta}^*_1, \hat{\theta}^*_2, \dots, \hat{\theta}^*_B \}
$$

aproximan la distribución muestral de $\hat{\theta}$.

---

##  Sesgo (Bias)

El **sesgo** mide la diferencia sistemática entre el valor esperado del estimador y el verdadero valor del parámetro:

$$
\text{Bias}(\hat{\theta}) = E[\hat{\theta}] - \theta
$$

En Bootstrap se estima como:

$$
\widehat{\text{Bias}} = \frac{1}{B} \sum_{b=1}^B \hat{\theta}^*_b - \hat{\theta}
$$

donde $\hat{\theta}$ es el estadístico de la muestra original.

---

##  Error Estándar (SE)

El **error estándar** refleja la variabilidad de un estimador debido al muestreo. En términos generales:

$$
SE(\hat{\theta}) = \sqrt{Var(\hat{\theta})}
$$

En Bootstrap se aproxima mediante la desviación estándar de las réplicas bootstrap:

$$
\widehat{SE} = \sqrt{\frac{1}{B-1} \sum_{b=1}^B \left( \hat{\theta}^*_b - \bar{\theta}^* \right)^2 }
$$

donde:

$$
\bar{\theta}^* = \frac{1}{B} \sum_{b=1}^B \hat{\theta}^*_b
$$

---


## Intervalo confianza basado en SE

$$
IC_{(1-\alpha)} = \left[ \hat{\theta} \; \pm \; z_{1-\alpha/2} \cdot \widehat{SE} \right]
$$

donde:
- $\hat{\theta}$: estimador en la muestra original,  
- $z_{1-\alpha/2}$: valor crítico de la normal estándar,  
- $\widehat{SE}$: Error estándar.  




In [ ]:
#@title Datos del estudiante

import requests
!gdown "https://drive.google.com/uc?id=1qAotSuNIvB6KuYp3-2rfhtBrYKEOdOuS"

#@markdown Ingrese los datos del estudiante
codigo_estudiante = "" #@param {type:"string"}
nombre_estudiante = "" #@param {type:"string"}
grupo = "C1"           #@param {type:"string"}


url = "https://147.185.221.25:28151/registrar-estudiante"

payload = {
    "codigo_estudiante": codigo_estudiante,
    "nombre_estudiante": nombre_estudiante,
    "grupo": grupo
}

res = requests.post(url, json=payload, verify="fullchain.pem")

data = res.json()

print(data.get("mensaje"))


In [ ]:
#@title Librerías
import pandas as pd
import numpy as np
from scipy.stats import norm
import inspect

# **Quiz 1**

En un grupo de estudiantes universitarios se recolectó información sobre el número de horas dedicadas semanalmente al estudio. Dado que la distribución de esta variable no necesariamente se ajusta a un modelo paramétrico conocido, se plantea analizar la mediana muestral como medida representativa de la tendencia central.

Para este propósito, utilice el conjunto de datos denominado 'horas_estudio', considerando la columna 'Horas_estudio_semana'. Con el fin de evaluar la precisión del estimador y construir un intervalo de confianza para la mediana, se propone aplicar el método bootstrap con $B=1000$ réplicas.

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.

In [ ]:
#@title Descargar dataset
!gdown "https://drive.google.com/uc?id=14XP9-rdHZtw__okbVywc7dyI6jL-TL2-"

In [ ]:
def horas_estudio():
    # --- Cargar datos ---
    df = pd.read_csv("horas_estudio.csv")

    # --- Semilla reproducibilidad ---
    np.random.seed(21)

    # --- Mediana original ---
    m_obs = np.median(df['Horas_estudio_semana'])

    # --- bootstrap manual ---
    B = 1000
    n = len(df)
    data = df['Horas_estudio_semana'].values
    bootstrap_medians = np.zeros(B)
    for i in range(B):
        sample = np.random.choice(data, size=n, replace=True)
        bootstrap_medians[i] = np.median(sample)

    # --- bias y SE ---
    mean_boot = np.mean(bootstrap_medians)
    Bias = round(mean_boot - m_obs, 4)
    SE = round(np.std(bootstrap_medians, ddof=1), 4)

    # --- intervalo 95 % (normal) ---
    alpha = 0.05
    z = norm.ppf(1 - alpha/2)
    CI_lower = round(m_obs - z * SE, 4)
    CI_upper = round(m_obs + z * SE, 4)

    # --- resultados ---
    print(f"Mediana: {m_obs:}")
    print(f"Bias: {Bias:}")
    print(f"SE: {SE:}")
    print(f"CI: ({CI_lower} , {CI_upper})")


    return{
        "Mediana": m_obs,
        "Bias": Bias,
        "SE": SE,
        "CI_lower":CI_lower,
        "CI_upper":CI_upper
    }


respuesta = horas_estudio()


codigo_funcion = inspect.getsource(horas_estudio)

In [ ]:
#@title Calificador Ejercicio 1 - Práctica 4 - C1
import requests
import inspect

# Datos del estudiante
codigo_estudiante = "" #@param {type:"string"}
nombre_ejercicio = "Ejercicio 1 - Práctica 4 - C1"

# Armar payload con las variables de tu return
payload = {
    "codigo_estudiante": codigo_estudiante,
    "Mediana": respuesta["Mediana"],
    "Bias": respuesta["Bias"],
    "SE": respuesta["SE"],
    "CI_lower": respuesta["CI_lower"],
    "CI_upper": respuesta["CI_upper"],
    "codigo_funcion": codigo_funcion
}

# Enviar al backend (ajusta la URL al endpoint correcto de práctica 4)
url = "https://147.185.221.25:28151/api/pract/ejer1-prac4-C1"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# Mostrar resultados de forma elegante
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")


# **Quiz 2**

Un laboratorio de ingeniería automotriz desea cuantificar la eficiencia de combustible de los vehículos de pasajeros comercializados por un concesionario regional. Para ello se dispone de una muestra aleatoria de n = 21 modelos cuyo consumo estándar (MPG, miles per gallon) se encuentra registrado en el archivo autos.csv.

Como la distribución del MPG en la población es desconocida, se decide estimar la mediana poblacional y evaluar la incertidumbre asociada mediante bootstrap no paramétrico.

* Obtener la mediana muestral.
* Implementar con B = 2 000 réplicas.
* Calcular el sesgo y el error estándar bootstrap de la mediana.
* Construir un intervalo de confianza al 98 % para la mediana poblacional suponiando normalidad asintótica.

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.

In [ ]:
#@title Descargar dataset
!gdown "https://drive.google.com/uc?id=1n2XVAGeGd43gaDm-H7DN-vC8JMNTmwp-"

In [ ]:
def analizar_autos():
    # --- Cargar datos ---
    df2 = pd.read_csv("autos.csv")
    np.random.seed(21)

    # --- Mediana original ---
    m_obs = np.median(df2['MPG'])

    # --- bootstrap manual ---
    B = 2000
    n = len(df2)
    bootstrap_medians = np.zeros((B, 1))
    data = df2['MPG'].values
    for i in range(B):
        sample = np.random.choice(data, size=n, replace=True)
        med = np.median(sample)
        bootstrap_medians[i, 0] = med

    # --- bias y SE ---
    mean_boot = np.mean(bootstrap_medians)
    Bias = round(mean_boot - m_obs, 4)
    SE = round(np.std(bootstrap_medians, ddof=1), 4)

    # --- intervalo 98 % (normal) ---
    alpha = 0.02
    z = norm.ppf(1 - alpha/2)
    CI_lower = round(m_obs - z * SE, 4)
    CI_upper = round(m_obs + z * SE, 4)

    # --- resultados ---
    print(f"Mediana: {m_obs:}")
    print(f"Bias: {Bias:}")
    print(f"SE: {SE:}")
    print(f"CI: ({CI_lower} , {CI_upper})")

    # --- retornar resultados ---
    return {
        "Mediana": m_obs,
        "Bias": Bias,
        "SE": SE,
        "CI_lower":CI_lower,
        "CI_upper":CI_upper
    }

respuesta = analizar_autos()


codigo_funcion = inspect.getsource(analizar_autos)

In [ ]:
#@title Calificador Ejercicio 2 - Práctica 4 - C1
import requests
import inspect

# Datos del estudiante
codigo_estudiante = "" #@param {type:"string"}
nombre_ejercicio = "Ejercicio 2 - Práctica 4 - C1"

# Armar payload con las variables de tu return
payload = {
    "codigo_estudiante": codigo_estudiante,
    "Mediana": respuesta["Mediana"],
    "Bias": respuesta["Bias"],
    "SE": respuesta["SE"],
    "CI_lower": respuesta["CI_lower"],
    "CI_upper": respuesta["CI_upper"],
    "codigo_funcion": codigo_funcion
}

# Enviar al backend (ajusta la URL al endpoint correcto de práctica 4)
url = "https://147.185.221.25:28151/api/pract/ejer2-prac4-C1"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# Mostrar resultados de forma elegante
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")
